In [1]:
# Enable autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import pickle
from pathlib import Path
import numpy as np
import pgeocode
import datetime
import re
from datetime import datetime

pd.set_option('max_columns',None)

# Import Raw Data

Import Raw Data from Excel files

In [3]:
raw_path = Path('./data/export_raw_rech_2016_2020.pkl.bz2')

if raw_path.exists():
    df_raw = pd.read_pickle(raw_path)
else:
    files = [
        './data/raw/export_GJ2016.xlsx', 
        './data/raw/export_GJ2017.xlsx', 
        './data/raw/export_GJ2018.xlsx',
        './data/raw/export_GJ2019.xlsx',
        './data/raw/export_GJ2020.xlsx'
    ]

    df_list = []

    for file in tqdm(files):
        print('Importing file "{}"'.format(file))
        df_tmp = pd.read_excel(file)
        df_list.append(df_tmp)
    
    df_raw = pd.concat(df_list)
    df_raw.to_pickle('./data/export_raw_rech_2016_2020.pkl.bz2')

display(df_raw.head())

,ID,IMP_ID,GJ,IMPORT,MUFO_REFERENZ_NR,BARCODE_NR,VERANST_SEGMENT,REKLA_JN,VG_DATUM_VON,VG_ORT,VG_RAUM,NUTZFALL_VON_DT,NUTZFALL_RAUM,MUSIKLEITER_NAME,KAPELLE_NAME,TARIF_NR,TARIF_BEZ,NUTZFALL_NR,VG_INKASSO,INKASSO_NETTO,INKASSO_BRUTTO,VERANST_GESCHAEFTSZEICHEN,VERANST_NAME,VERANST_STRASSE,VERANST_PLZ,VERANST_ORT,NUTZLIZNEHM_GESCHAEFTSZEICHEN,NUTZLIZNEHM_NAME,NUTZLIZNEHM_VORNAME,NUTZLIZNEHM_STRASSE,NUTZLIZNEHM_PLZ,NUTZLIZNEHM_ORT
0,10381586,24643,2016,ABRE U INKA 2016,NaN,5207017125,3.0,NONRekla,20161111,VALLEY,WERKSTAETTE BRUNNER,20161111.0,WERKSTAETTE BRUNNER,NaN,NaN,1201,U-V II. 1(+II 2),26818225,69.15,NaN,NaN,922236900,BLASKAPELLE UNTERDARCHING E.V.,HAFNERSTR. 8,83626,VALLEY,1510423300,BAYERISCHER BLASMUSIKVERBAND,NaN,SANDSTR. 31,80335,MUENCHEN
1,10438107,24643,2016,ABRE U INKA 2016,NaN,5206736823,3.0,NONRekla,20161030,HAGEN,CIRCUS VOYAGE,20161030.0,CIRCUS VOYAGE,NaN,NaN,519,VK I 3 ZIRKUSUNTERNEHMEN (MUSIKER),27309955,50.72,NaN,NaN,375311300,ALOIS SPINDLER,GOESSELNHOF 7,58093,HAGEN,375311300,SPINDLER,ALOIS,GOESSELNHOF 7,58093,HAGEN
2,10010478,24643,2016,ABRE U INKA 2016,NaN,5082429877,2.0,NONRekla,20140626,OSTSEEBAD HERINGSDORF,USEDOMER BRAUHAUS/OSTSEERESIDENZ HERINGSDORF,20140626.0,USEDOMER BRAUHAUS/OSTSEERESIDENZ HERINGSDORF,NaN,NaN,1087,U-V II. 1 (+II 2),21292233,46.28,NaN,NaN,232362000,OSTSEERESIDENZ SEEBAD-HERINGSD. GMBH & CO.INVE...,SEESTR. 41,17424,OSTSEEBAD HERINGSDORF,232362000,OSTSEERESIDENZ SEEBAD-HERINGSD. GMBH & CO.INVE...,NaN,SEESTR. 41,17424,OSTSEEBAD HERINGSDORF
3,10056951,24643,2016,ABRE U INKA 2016,NaN,5085935333,2.0,NONRekla,20150425,MAULBRONN,SAENGERTREFF GV EINTRACHT SCHMIE,20150425.0,SAENGERTREFF GV EINTRACHT SCHMIE,NaN,NaN,892,U-K (MUSIKER),21740273,45.00,NaN,NaN,1136208900,GESANGVEREIN EINTRACHT SCHMIE 1891 E.V.,LIENZINGER STR. 17,75433,MAULBRONN,1510290100,DEUTSCHER CHORVERBAND,NaN,ALTE JAKOBSTR. 149,10969,BERLIN
4,10414299,24643,2016,ABRE U INKA 2016,NaN,6704114029,3.0,NONRekla,20151010,WEILMUENSTER,DGH LANGENBACH,20151010.0,DGH LANGENBACH,NaN,NaN,1201,U-V II. 1(+II 2),22926191,78.82,NaN,NaN,1211507100,KYFFHAEUSER KAMERADSCHAFT LANGENBACH .,LANGGASSE 47,35789,WEILMUENSTER,1211507100,KYFFHAEUSER KAMERADSCHAFT LANGENBACH,NaN,LANGGASSE 47,35789,WEILMUENSTER


In [4]:
df = df_raw.copy()
display(df_raw.isna().sum())

ID                                     0
IMP_ID                                 0
GJ                                     0
IMPORT                                 0
MUFO_REFERENZ_NR                 3727979
BARCODE_NR                             0
VERANST_SEGMENT                   314926
REKLA_JN                               0
VG_DATUM_VON                           0
VG_ORT                                10
VG_RAUM                              856
NUTZFALL_VON_DT                      713
NUTZFALL_RAUM                     315781
MUSIKLEITER_NAME                 2422606
KAPELLE_NAME                     1720166
TARIF_NR                               0
TARIF_BEZ                              0
NUTZFALL_NR                            0
VG_INKASSO                             0
INKASSO_NETTO                    2690836
INKASSO_BRUTTO                   2690836
VERANST_GESCHAEFTSZEICHEN              0
VERANST_NAME                         372
VERANST_STRASSE                    24033
VERANST_PLZ     

# Clean Raw Data

## Remove Segment 1, 13 and entries older than 2014

In [5]:
# VERANST_SEGMENT is between 2 and 12
df=df[(~df['VERANST_SEGMENT'].isnull()) & (~df['VERANST_SEGMENT'].isin([1,13]))]

#VG_DATUM_VON - convert to datetime and drop entries with year < 2014
  
df['VG_DATUM_VON'] = pd.to_datetime(df['VG_DATUM_VON'], format='%Y%m%d')
df=df[df['VG_DATUM_VON']>'2014-01-01']

display(df.shape[0]/df_raw.shape[0])

0.8385066937830151

In [6]:
df = df.reset_index(drop=True)
df[df.index.duplicated()]

,ID,IMP_ID,GJ,IMPORT,MUFO_REFERENZ_NR,BARCODE_NR,VERANST_SEGMENT,REKLA_JN,VG_DATUM_VON,VG_ORT,VG_RAUM,NUTZFALL_VON_DT,NUTZFALL_RAUM,MUSIKLEITER_NAME,KAPELLE_NAME,TARIF_NR,TARIF_BEZ,NUTZFALL_NR,VG_INKASSO,INKASSO_NETTO,INKASSO_BRUTTO,VERANST_GESCHAEFTSZEICHEN,VERANST_NAME,VERANST_STRASSE,VERANST_PLZ,VERANST_ORT,NUTZLIZNEHM_GESCHAEFTSZEICHEN,NUTZLIZNEHM_NAME,NUTZLIZNEHM_VORNAME,NUTZLIZNEHM_STRASSE,NUTZLIZNEHM_PLZ,NUTZLIZNEHM_ORT


## Text cleansing function

In [7]:
def clean_tarif(df_row):
    result = str(df_row)
    if result == "U-V III.1" :
        result = "U-V III. 1"
    elif result == "U-V III.3" :
        result = "U-V III. 3"
    elif result == "U-V II 1(+II 2A) KARAOKE" :
        result = "U-V II. 1 (+II 2A) KARAOKE"
    elif result == "U-V II. 1(+II 2)" :
        result = "U-V II. 1 (+II 2)"
    elif result == "U-ST I (MUSIKER)" :
        result = "U-ST I. (MUSIKER)"
    
    return result

print("Number of unique TARIF_BEZ before transformation {}".format(len(df.TARIF_BEZ.unique()))) 
df['TARIF_BEZ'] = df['TARIF_BEZ'].apply(clean_tarif)
print("Number of unique TARIF_BEZ after transformation {}".format(len(df.TARIF_BEZ.unique()))) 


Number of unique TARIF_BEZ before transformation 97
Number of unique TARIF_BEZ after transformation 92


In [8]:
def clean_text(df_row):
    """
    Cleans text data
        * removes leading and trailing special characters
        * replaces empty string, (KEINE ANGABE) and single " with np.Nan

    Parameters
    ----------
    df_row : Series element

    Returns
    -------
    string
    """
    result = str(df_row)
    result = result.strip(" .*:/-,+&?;")
    if result in ['nan','','(KEINE ANGABE)', '"']:
        return np.nan

    return result


text_columns = [
    "VG_RAUM", "MUSIKLEITER_NAME", "KAPELLE_NAME", "VERANST_NAME", "VERANST_STRASSE", "VERANST_ORT","VERANST_PLZ"
]

for col in tqdm(text_columns):
    df[col] = df[col].apply(clean_text)

display((df[text_columns] != df[text_columns]).mean(axis=0))

#with pd.option_context("display.max_rows", 100):
#    display(df[col].value_counts()[:100])

  0%|          | 0/7 [00:00<?, ?it/s]

VG_RAUM             0.000045
MUSIKLEITER_NAME    0.613268
KAPELLE_NAME        0.407166
VERANST_NAME        0.000116
VERANST_STRASSE     0.006167
VERANST_ORT         0.000189
VERANST_PLZ         0.000354
dtype: float64

## Replace Postcode with City Name

In [9]:
plz_matcher = re.compile('^([0-9]{5})$')
nomi = pgeocode.Nominatim('de')


# clean VG_ORT
def clean_place(df_row):
    """
    Replace

    Parameters
    ----------
    df_row : Series element

    Returns
    -------
    string
    """
    result = str(df_row)
    if plz_matcher.match(result):
        geo_result = nomi.query_postal_code(result)
        result = geo_result.place_name.split(',')[0]
        result = result.replace('ß', 'SS')
        result = result.upper()
        result = result.replace('Ä', 'AE')
        result = result.replace('Ö', 'OE')
        result = result.replace('Ü', 'UE')
        print(result)

    return result


df['VG_ORT'] = df['VG_ORT'].apply(clean_text).fillna('').apply(clean_place).apply(clean_text)

display((df[['VG_ORT']] != df[['VG_ORT']]).mean(axis=0))

LINDAU (BODENSEE)
LINDAU (BODENSEE)
WOLTERSDORF
BAD SCHOENBORN
LINDAU (BODENSEE)
BAD SCHOENBORN
LINDAU (BODENSEE)
LINDAU (BODENSEE)


VG_ORT    0.000063
dtype: float64

# Prepare for Feature Generation

## Create LOCATION column

In [10]:
df['LOCATION']=df['VG_ORT'].fillna('')+' ' + df['VG_RAUM'].fillna('')
df['LOCATION']=df['LOCATION'].apply(clean_text)

display(df['LOCATION'].head(10))
display(df[['VG_ORT','VG_RAUM','LOCATION']].isna().sum())


0                           VALLEY WERKSTAETTE BRUNNER
1                                  HAGEN CIRCUS VOYAGE
2    OSTSEEBAD HERINGSDORF USEDOMER BRAUHAUS/OSTSEE...
3           MAULBRONN SAENGERTREFF GV EINTRACHT SCHMIE
4                          WEILMUENSTER DGH LANGENBACH
5             PRIEN A. CHIEMSEE FERIENHAUS AM ROSENECK
6                                 KONSTANZ KULTURLADEN
7                OBERHACHING BUERGERSAAL BEIM FORSTNER
8                    KELSTERBACH IM FREIEN KELSTERBACH
9              ROTTENBURG AM NECKAR TURNHALLE SEEBRONN
Name: LOCATION, dtype: object

VG_ORT      204
VG_RAUM     144
LOCATION    142
dtype: int64

## Create BAND column

In [11]:
#Create BAND Column
df['BAND']=df['KAPELLE_NAME'].fillna(df['MUSIKLEITER_NAME'])

display(df['BAND'].dropna().head(10))
display(df[['KAPELLE_NAME','BAND']].isna().sum())

445        MV BLEICHHEIM EV
446    MUSIKGRUPPE HOH'WALD
447              MARGA BACH
448                   HANAK
449        VINCENT&FERNANDO
450          JOERG SOLLBACH
451               SCHMUTZKI
452          GIZMO VARILLAS
453    BOMBSHELLS DEAFEN GO
454          MARGITA PEYROT
Name: BAND, dtype: object

KAPELLE_NAME    1308080
BAND            1283443
dtype: int64

## Create PROMOTER column

In [12]:
#Create PROMOTER Column
df['PROMOTER']=df['VERANST_NAME'].fillna('')+' ' + df['VERANST_PLZ'].fillna('')
df['PROMOTER']=df['PROMOTER'].apply(clean_text)

display(df['PROMOTER'].head(10))
display(df[['PROMOTER','VERANST_NAME','VERANST_PLZ']].isna().sum())


0                  BLASKAPELLE UNTERDARCHING E.V 83626
1                                 ALOIS SPINDLER 58093
2    OSTSEERESIDENZ SEEBAD-HERINGSD. GMBH & CO.INVE...
3         GESANGVEREIN EINTRACHT SCHMIE 1891 E.V 75433
4           KYFFHAEUSER KAMERADSCHAFT LANGENBACH 35789
5                                  MARTIN HIRNER 80469
6                       KULTURLADEN KONSTANZ E.V 78467
7                      SARRE MUSIKPROJEKTE GGMBH 82064
8                      KANU CLUB KELSTERBACH E.V 65451
9                 SCHUETZENGESELLSCHAFT SEEBRONN 72108
Name: PROMOTER, dtype: object

PROMOTER         118
VERANST_NAME     373
VERANST_PLZ     1137
dtype: int64

# Clean VG_INKASSO

In [13]:
df.loc[df.VG_INKASSO == 0, 'VG_INKASSO'] = np.nan

#display(df.loc[df.VG_INKASSO.isnull()])
display('NaN values in df_raw : {}, NaN values in df: {}'.format(df_raw['VG_INKASSO'].isna().sum(), df['VG_INKASSO'].isna().sum()))

'NaN values in df_raw : 0, NaN values in df: 4'

# Drop Duplicates

In [14]:
df = df.dropna(subset=['VG_INKASSO'])
df = df.drop_duplicates(subset=['ID']).set_index('ID', verify_integrity=True).sort_index()

#convert certain columns to 'category'

df['IMPORT'] = df['IMPORT'].astype('category')
df['REKLA_JN'] = df['REKLA_JN'].astype('category')
df['TARIF_BEZ'] = df['TARIF_BEZ'].astype('category')
df['VG_ORT'] = df['VG_ORT'].astype('category')
df['VG_RAUM'] = df['VG_RAUM'].astype('category')

In [15]:
display(df.shape[0]/df_raw.shape[0])
print(df.isna().sum())

0.8226364785008891

IMP_ID                                 0
GJ                                     0
IMPORT                                 0
MUFO_REFERENZ_NR                 3068567
BARCODE_NR                             0
VERANST_SEGMENT                        0
REKLA_JN                               0
VG_DATUM_VON                           0
VG_ORT                               204
VG_RAUM                              144
NUTZFALL_VON_DT                        0
NUTZFALL_RAUM                        142
MUSIKLEITER_NAME                 1939352
KAPELLE_NAME                     1297000
TARIF_NR                               0
TARIF_BEZ                              0
NUTZFALL_NR                            0
VG_INKASSO                             0
INKASSO_NETTO                    2240608
INKASSO_BRUTTO                   2240608
VERANST_GESCHAEFTSZEICHEN              0
VERANST_NAME                         373
VERANST_STRASSE                    19437
VERANST_PLZ                         1106
VERANST_ORT     

# Store Prepared Data

Store prepared data in a pickle file for further usage

In [16]:
df.to_pickle('./data/export_rech_2016_2020.pkl.bz2', protocol=4)